In [ ]:
// Updated Frontend MVP for Passport/Visa Photo App (React + Tailwind + Webcam + Country Rules)

import React, { useState, useRef } from 'react';
import { Button } from '@/components/ui/button';
import { Card, CardContent } from '@/components/ui/card';
import { Input } from '@/components/ui/input';

const countryRequirements = {
  USA: { width: 600, height: 600, background: 'white' },
  France: { width: 827, height: 1063, background: 'white' },
  Pakistan: { width: 413, height: 531, background: 'blue' },
  UK: { width: 900, height: 1200, background: 'white' },
  Canada: { width: 420, height: 540, background: 'white' },
  Australia: { width: 354, height: 472, background: 'white' },
  Germany: { width: 600, height: 600, background: 'white' },
  SaudiArabia: { width: 200, height: 200, background: 'white' },
  UAE: { width: 600, height: 600, background: 'white' },
  China: { width: 354, height: 472, background: 'white' },
  Turkey: { width: 500, height: 500, background: 'white' },
  Malaysia: { width: 413, height: 531, background: 'white' },
  Iran: { width: 400, height: 600, background: 'white' },
  Russia: { width: 420, height: 540, background: 'white' },
  Italy: { width: 827, height: 1063, background: 'white' },
  Spain: { width: 827, height: 1063, background: 'white' },
  Japan: { width: 450, height: 600, background: 'white' },
  Korea: { width: 413, height: 531, background: 'white' }
};

export default function PassportPhotoApp() {
  const [capturedImage, setCapturedImage] = useState(null);
  const [country, setCountry] = useState('USA');
  const [quantity, setQuantity] = useState(1);
  const [paymentMethod, setPaymentMethod] = useState('card');
  const [language, setLanguage] = useState('en');
  const [processedImage, setProcessedImage] = useState(null);
  const videoRef = useRef(null);
  const canvasRef = useRef(null);

  const startCamera = async () => {
    const stream = await navigator.mediaDevices.getUserMedia({ video: true });
    videoRef.current.srcObject = stream;
    videoRef.current.play();
  };

  const capturePhoto = () => {
    const context = canvasRef.current.getContext('2d');
    const { width, height } = countryRequirements[country];
    canvasRef.current.width = width;
    canvasRef.current.height = height;
    context.drawImage(videoRef.current, 0, 0, width, height);
    const dataUrl = canvasRef.current.toDataURL('image/jpeg');
    setCapturedImage(dataUrl);
  };

  const handleSubmit = async () => {
    const { background, width, height } = countryRequirements[country];
    const response = await fetch('http://localhost:5000/process', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ image: capturedImage, background, width, height, quantity })
    });
    const data = await response.json();
    setProcessedImage(data.processed);
  };

  const handlePayment = async () => {
    const response = await fetch('http://localhost:5000/pay', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ method: paymentMethod, amount: quantity * 50 })
    });
    const result = await response.json();
    alert(result.message);
  };

  return (
    <div className="max-w-3xl mx-auto p-4 space-y-4">
      <h1 className="text-2xl font-bold">
        {language === 'en' ? 'Passport/Visa Photo App (Pakistan)' : 'پاسپورٹ / ویزا فوٹو ایپ'}
      </h1>

      <div className="flex items-center gap-4">
        <label>{language === 'en' ? 'Language:' : 'زبان:'}</label>
        <select value={language} onChange={e => setLanguage(e.target.value)} className="p-2 rounded">
          <option value="en">English</option>
          <option value="ur">اردو</option>
        </select>
      </div>

      <Card>
        <CardContent className="space-y-2">
          <video ref={videoRef} className="w-full" autoPlay muted />
          <canvas ref={canvasRef} className="hidden" />
          <Button onClick={startCamera}>{language === 'en' ? 'Start Camera' : 'کیمرہ شروع کریں'}</Button>
          <Button onClick={capturePhoto}>{language === 'en' ? 'Capture Photo' : 'تصویر لیں'}</Button>
        </CardContent>
      </Card>

      <Card>
        <CardContent className="space-y-2">
          <label>{language === 'en' ? 'Select Country:' : 'ملک منتخب کریں:'}</label>
          <select onChange={e => setCountry(e.target.value)} className="p-2 rounded">
            {Object.keys(countryRequirements).map(c => (
              <option key={c} value={c}>{c}</option>
            ))}
          </select>
          <p className="text-sm text-gray-500">
            {language === 'en' ? 'Required background:' : 'ضروری پس منظر:'} {countryRequirements[country].background}
          </p>
          <Input type="number" min="1" max="15" value={quantity} onChange={e => setQuantity(Number(e.target.value))} />
          <Button onClick={handleSubmit}>{language === 'en' ? 'Process Photo' : 'تصویر تیار کریں'}</Button>
        </CardContent>
      </Card>

      {capturedImage && <img src={capturedImage} alt="Captured" className="w-full max-w-xs" />}
      {processedImage && <img src={processedImage} alt="Processed" className="w-full max-w-xs" />}

      <Card>
        <CardContent className="space-y-2">
          <label>{language === 'en' ? 'Payment Method:' : 'ادائیگی کا طریقہ:'}</label>
          <select onChange={e => setPaymentMethod(e.target.value)} className="p-2 rounded">
            <option value="card">{language === 'en' ? 'Bank Card' : 'بینک کارڈ'}</option>
            <option value="easypaisa">EasyPaisa</option>
            <option value="jazzcash">JazzCash</option>
          </select>
          <Button onClick={handlePayment}>{language === 'en' ? 'Pay Now' : 'ابھی ادائیگی کریں'}</Button>
        </CardContent>
      </Card>
    </div>
  );
}
